In [1]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import tensorflow as tf

# from pykospacing import Spacing
# import re

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
train_df = pd.read_csv(r'./data/korean-hate-speech-detection/train.hate.csv')

train_text = open(r'./data/korean-hate-speech-detection/train.news_title.txt', 'r')
train_texts = train_text.readlines()

train_text_list = []
for line in train_texts:
    train_text_list.append(line)
train_text.close()

train_df['text'] = train_text_list
train_df

In [ ]:
lengh = []
for i in train_df['comments']:
    lengh.append(len(i))
max(lengh)

In [54]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()  # 소문자 변경
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 기본 불용어 불러오기
korean_stopwords_path = "data/stopwords-ko.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

# 불용어 처리 함수 수정
def remove_stopwords(text, stopwords):
    words = text.split()  # 문장을 단어로 분리
    filtered_text = [word for word in words if word not in stopwords]  # 단어가 불용어 목록에 없는 경우만 추가
    return ' '.join(filtered_text)  # 필터링된 단어들 다시 조합

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(train_df))):
    comment_text = train_df.loc[i, 'comments']
    newstile_text = train_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    train_df.loc[i, 'processed_comments'] = cleaned_comment
    train_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [ ]:
train_df.loc[train_df['label'] == 'none', 'labels'] = 0
train_df.loc[train_df['label'] == 'offensive', 'labels'] = 1
train_df.loc[train_df['label'] == 'hate', 'labels'] = 2

train_df['labels'] = train_df['labels'].astype(int)
train_df.info()

In [70]:
train_df.to_csv('./data/train_df_processed.csv', index=False, encoding='utf-8-sig')

In [ ]:
dev_df = pd.read_csv(r'./data/korean-hate-speech-detection/dev.hate.csv')

dev_text = open(r'./data/korean-hate-speech-detection/dev.news_title.txt', 'r')
dev_texts = dev_text.readlines()

dev_text_list = []
for line in dev_texts:
    dev_text_list.append(line)
dev_text.close

dev_df['text'] = dev_text_list
dev_df

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(dev_df))):
    comment_text = dev_df.loc[i, 'comments']
    newstile_text = dev_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    dev_df.loc[i, 'processed_comments'] = cleaned_comment
    dev_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [88]:
dev_df.loc[dev_df['label'] == 'none', 'labels'] = 0
dev_df.loc[dev_df['label'] == 'offensive', 'labels'] = 1
dev_df.loc[dev_df['label'] == 'hate', 'labels'] = 2
dev_df['labels'] = dev_df['labels'].astype(int)

dev_df.to_csv('./data/dev_df_processed.csv', index=False, encoding='utf-8-sig')

In [ ]:
dev_df = pd.read_csv('./data/dev_df_processed.csv')
dev_df

In [ ]:
test_df = pd.read_csv(r'./data/korean-hate-speech-detection/test.hate.no_label.csv')

test_text = open(r'./data/korean-hate-speech-detection/test.news_title.txt')
test_texts = test_text.readlines()
test_text_list = []
for line in test_texts:
    test_text_list.append(line)

test_df['text'] = test_text_list
test_df

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(test_df))):
    comment_text = test_df.loc[i, 'comments']
    newstile_text = test_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    test_df.loc[i, 'processed_comments'] = cleaned_comment
    test_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [67]:
test_df.to_csv('./data/test_df_processed.csv', index=False, encoding='utf-8-sig')

In [2]:
train_df = pd.read_csv(r'./data/train_df_processed.csv')
dev_df = pd.read_csv(r'./data/dev_df_processed.csv')
test_df = pd.read_csv(r'./data/test_df_processed.csv')

In [3]:
model_name = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/p

In [4]:
with tf.device('/GPU:0'):
    tensor = tf.constant([1.0, 2.0, 3.0])
print("device:", tensor.device)

device: /job:localhost/replica:0/task:0/device:GPU:0


2024-11-01 16:26:14.588878: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-11-01 16:26:14.588892: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-11-01 16:26:14.588899: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-11-01 16:26:14.589080: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-01 16:26:14.589091: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
train_df = train_df.dropna()

In [68]:
# 데이터 토크나이징

tokenized_train_comment = tokenizer(
    list(train_df['processed_comments']),
    return_tensors='tf',
    # max_length=128,
    padding='max_length',
    truncation=True,          # max_length 초과 토큰 truncate
    add_special_tokens=True,  # special token 추가
)

tokenized_train_title = tokenizer(
    list(train_df['processed_newstitle']),
    return_tensors='tf',
    # max_length=128,
    padding='max_length',
    truncation=True,          # max_length 초과 토큰 truncate
    add_special_tokens=True,  # special token 추가
)

tokenized_dev_comment = tokenizer(
    list(dev_df['processed_comments']),
    return_tensors='tf',
    # max_length=128,
    padding='max_length',
    truncation=True,          # max_length 초과 토큰 truncate
    add_special_tokens=True,  # special token 추가
)

tokenized_dev_title = tokenizer(
    list(dev_df['processed_newstitle']),
    return_tensors='tf',
    # max_length=128,
    padding='max_length',
    truncation=True,          # max_length 초과 토큰 truncate
    add_special_tokens=True,  # special token 추가
)

In [69]:
tokenized_train_comment

{'input_ids': <tf.Tensor: shape=(7893, 512), dtype=int32, numpy=
array([[    2,  9008, 15136, ...,     0,     0,     0],
       [    2,  7991,  8065, ...,     0,     0,     0],
       [    2, 11026, 11826, ...,     0,     0,     0],
       ...,
       [    2,  9196, 33024, ...,     0,     0,     0],
       [    2,  9196, 22597, ...,     0,     0,     0],
       [    2, 24810,   966, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(7893, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(7893, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=in

In [70]:
tokenized_dev_comment

{'input_ids': <tf.Tensor: shape=(471, 512), dtype=int32, numpy=
array([[    2, 53063,  9237, ...,     0,     0,     0],
       [    2, 51783,  4194, ...,     0,     0,     0],
       [    2,  8926,  9238, ...,     0,     0,     0],
       ...,
       [    2,  8205, 28149, ...,     0,     0,     0],
       [    2, 16679,  4826, ...,     0,     0,     0],
       [    2, 10879, 13827, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(471, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(471, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32

In [71]:
# CLS 토큰 벡터 추출
train_comment_embedding = tf.reshape(tokenized_train_comment['input_ids'], (7893,-1))
train_title_embedding = tf.reshape(tokenized_train_title['input_ids'], (-1,))
dev_comment_embedding = tf.reshape(tokenized_dev_comment['input_ids'], (471,-1))
dev_title_embedding = tf.reshape(tokenized_dev_title['input_ids'], (-1,))

In [49]:
train_df['labels'] = train_df['labels'].astype(int)

In [43]:
# 2D 배열로 변환
# X_train = tf.reshape(train_comment_embedding, (-1, 1))
# X_test = tf.reshape(dev_comment_embedding, (-1, 1))     # 테스트 데이터도 2D로 변환
y_train = train_df['labels']
y_test = dev_df['labels']

In [72]:
train_comment_embedding

<tf.Tensor: shape=(7893, 512), dtype=int32, numpy=
array([[    2,  9008, 15136, ...,     0,     0,     0],
       [    2,  7991,  8065, ...,     0,     0,     0],
       [    2, 11026, 11826, ...,     0,     0,     0],
       ...,
       [    2,  9196, 33024, ...,     0,     0,     0],
       [    2,  9196, 22597, ...,     0,     0,     0],
       [    2, 24810,   966, ...,     0,     0,     0]], dtype=int32)>

In [73]:
dev_comment_embedding

<tf.Tensor: shape=(471, 512), dtype=int32, numpy=
array([[    2, 53063,  9237, ...,     0,     0,     0],
       [    2, 51783,  4194, ...,     0,     0,     0],
       [    2,  8926,  9238, ...,     0,     0,     0],
       ...,
       [    2,  8205, 28149, ...,     0,     0,     0],
       [    2, 16679,  4826, ...,     0,     0,     0],
       [    2, 10879, 13827, ...,     0,     0,     0]], dtype=int32)>

In [74]:
y_train

0       2
1       0
2       2
3       0
4       2
       ..
7891    0
7892    0
7893    0
7894    0
7895    0
Name: labels, Length: 7893, dtype: int64

In [84]:
# 로지스틱 회귀 모델 학습
logistic_model = LogisticRegression(max_iter=1000, multi_class='multinomial')
logistic_model.fit(train_comment_embedding, train_df['labels'])

# 예측 및 평가
y_pred = logistic_model.predict(dev_comment_embedding)
accuracy = accuracy_score(dev_df['labels'], y_pred)
f1 = f1_score(dev_df['labels'], y_pred, average='weighted')  # 다중 클래스 분류의 경우 'weighted' 또는 'macro' 사용
print("Accuracy:", accuracy)
print("F1 score:", f1)

Accuracy: 0.34394904458598724
F1 score: 0.2500706524399621


/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [85]:
# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_comment_embedding)
X_test_scaled = scaler.transform(dev_comment_embedding)

# 로지스틱 회귀 모델 초기화 및 학습
logistic_model = LogisticRegression(max_iter=2000, multi_class='multinomial', solver='saga')
logistic_model.fit(X_train_scaled, y_train)

# 테스트 데이터 예측
y_pred = logistic_model.predict(X_test_scaled)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print("Accuracy:", accuracy)
print("F1 Score:", f1)

Accuracy: 0.32908704883227174
F1 Score: 0.22604433570296728


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_comment_embedding, y_train)
y_pred = rf_model.predict(dev_comment_embedding)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')  # 다중 클래스 분류의 경우 'weighted' 또는 'macro' 사용
print("Accuracy:", accuracy)
print("F1 score:", f1)

RandomForestClassifier(random_state=42)